In [1]:
import keras
from keras.models import Sequential
from keras.layers import Activation,LSTM,Dense
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import pickle

Using TensorFlow backend.
/home/mayank/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/mayank/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df=pd.read_csv('data.csv')['text']

In [3]:
data=np.array(df)
print(data)

["All alone in my room think of you at a rate that is truly alarming  \nI keep grouping my memories of you in my head, I pretend that you want me  \nAnd I fall asleep and dream of alternate realities  \nAnd I put myself at ease by pretending that she still loves me  \n  \nAnd I can't stop thinking about you  \nAnd I can't stop thinking about you  \nYou never go on till I do  \nAnd I can't stop thinking about your love  \n  \nOh yeah  \n  \nCan't believe I could think that she would just follow me everywhere I go  \nI was wrestling with you in my dreams, and wake up making love to a pillow  \nAnd I fall asleep and dream of alternate realities  \nAnd I put myself at ease by pretending that she still loves me  \n  \nAnd I can't stop thinking about you  \nAnd I can't stop thinking about you  \nAnd you never go on till I do  \nAnd I can't stop, and I can't stop  \n  \nWhat I would give to have you look in my direction  \nAnd I'd give my life to somehow attract your attention  \nAnd I touch 

In [4]:
corpus=''
for ix in range(len(data)):
    corpus+=data[ix]

In [5]:
vocab=list(set(corpus))
char_ix={c:i for i,c in enumerate(vocab)}
ix_char={i:c for i,c in enumerate(vocab)}

In [6]:
f = open("char_ix.pkl","wb")
pickle.dump(char_ix,f)
f.close()

w = open("ix_char.pkl","wb")
pickle.dump(ix_char,w)
w.close()

In [7]:
pickle_in = open("ix_char.pkl","rb")
example_dict = pickle.load(pickle_in)

In [9]:
maxlen=40
vocab_size=len(vocab)

In [10]:
sentences=[]
next_char=[]
for i in range(len(corpus)-maxlen-1):
    sentences.append(corpus[i:i+maxlen])
    next_char.append(corpus[i+maxlen])

In [12]:
X=np.zeros((len(sentences),maxlen,vocab_size))
y=np.zeros((len(sentences),vocab_size))
for ix in range(len(sentences)):
    y[ix,char_ix[next_char[ix]]]=1
    for iy in range(maxlen):
        X[ix,iy,char_ix[sentences[ix][iy]]]=1

In [13]:
model=Sequential()
model.add(LSTM(128,input_shape=(maxlen,vocab_size)))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.summary()
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',  metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               101888    
_________________________________________________________________
dense_1 (Dense)              (None, 70)                9030      
_________________________________________________________________
activation_1 (Activation)    (None, 70)                0         
Total params: 110,918
Trainable params: 110,918
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X,y,epochs=5,batch_size=128, verbose=1)

Epoch 1/5
157991/157991 [==============================] - 113s 713us/step - loss: 1.8623 - acc: 0.4702
Epoch 2/5
157991/157991 [==============================] - 102s 643us/step - loss: 1.4298 - acc: 0.5762
Epoch 3/5
157991/157991 [==============================] - 116s 735us/step - loss: 1.3116 - acc: 0.6078
Epoch 4/5
157991/157991 [==============================] - 126s 799us/step - loss: 1.2457 - acc: 0.6260
Epoch 5/5
157991/157991 [==============================] - 128s 810us/step - loss: 1.2031 - acc: 0.6374


In [15]:
model.save('model.h5')

In [ ]:
import random
def predict():
    generated=''
    start_index=random.randint(0,len(corpus)-maxlen-1)
    sent=corpus[start_index:start_index+maxlen]
    generated+=sent
    for i in range(1900):
        x_sample=generated[i:i+maxlen]
        x=np.zeros((1,maxlen,vocab_size))
        for j in range(maxlen):
            x[0,j,char_ix[x_sample[j]]]=1
        probs=loaded_model.predict(x)
        probs=np.reshape(probs,probs.shape[1])
        ix=np.random.choice(range(vocab_size),p=probs.ravel())
        generated+=ix_char[ix]
        print(generated)

In [ ]:
predict()